In [1]:
import pandas as pd
import numpy as np
import pickle
import pdb

In [2]:
mta = pd.read_pickle("mta_daily_data_v1.pkl")

In [3]:
mta

,CA,UNIT,SCP,STATION,TURNSTILE,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES,WEEKDAY,WEEKDAY_INDEX,WEEK_OF_YEAR
1,A002,R051,02-00-00,59 ST,A002 R051 02-00-00 59 ST,2019-03-17,6981564,03/16/2019,6980704.0,860.0,Sunday,6,11
2,A002,R051,02-00-00,59 ST,A002 R051 02-00-00 59 ST,2019-03-18,6982134,03/17/2019,6981564.0,570.0,Monday,0,12
3,A002,R051,02-00-00,59 ST,A002 R051 02-00-00 59 ST,2019-03-19,6983646,03/18/2019,6982134.0,1512.0,Tuesday,1,12
4,A002,R051,02-00-00,59 ST,A002 R051 02-00-00 59 ST,2019-03-20,6985159,03/19/2019,6983646.0,1513.0,Wednesday,2,12
5,A002,R051,02-00-00,59 ST,A002 R051 02-00-00 59 ST,2019-03-21,6986742,03/20/2019,6985159.0,1583.0,Thursday,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372196,TRAM2,R469,00-05-01,RIT-ROOSEVELT,TRAM2 R469 00-05-01 RIT-ROOSEVELT,2019-05-27,5554,05/26/2019,5554.0,0.0,Monday,0,22
372197,TRAM2,R469,00-05-01,RIT-ROOSEVELT,TRAM2 R469 00-05-01 RIT-ROOSEVELT,2019-05-28,5554,05/27/2019,5554.0,0.0,Tuesday,1,22
372198,TRAM2,R469,00-05-01,RIT-ROOSEVELT,TRAM2 R469 00-05-01 RIT-ROOSEVELT,2019-05-29,5554,05/28/2019,5554.0,0.0,Wednesday,2,22
372199,TRAM2,R469,00-05-01,RIT-ROOSEVELT,TRAM2 R469 00-05-01 RIT-ROOSEVELT,2019-05-30,5554,05/29/2019,5554.0,0.0,Thursday,3,22


In [4]:
subway = pd.read_csv('nyc_open_data_subway.csv', usecols = ["NAME"])

In [5]:
subway

,NAME
0,Astor Pl
1,Canal St
2,50th St
3,Bergen St
4,Pennsylvania Ave
...,...
468,Coney Island - Stillwell Av
469,34th St - Hudson Yards
470,72nd St
471,86th St


In [6]:
mta_stations = pd.DataFrame(mta["STATION"].unique(), columns = ["STATION"])
mta_stations["STATION_CLEAN"] = mta_stations["STATION"].str.title()
mta_stations["STATION_CLEAN"] = mta_stations["STATION_CLEAN"].str.replace("/", "-")
mta_stations

,STATION,STATION_CLEAN
0,59 ST,59 St
1,5 AV/59 ST,5 Av-59 St
2,57 ST-7 AV,57 St-7 Av
3,49 ST,49 St
4,TIMES SQ-42 ST,Times Sq-42 St
...,...,...
374,EASTCHSTER/DYRE,Eastchster-Dyre
375,ST. GEORGE,St. George
376,TOMPKINSVILLE,Tompkinsville
377,RIT-MANHATTAN,Rit-Manhattan


In [7]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [8]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [9]:
#create function to add ordinal suffix to number
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])

In [10]:
#add ordinal suffix to street number (within string)
def insert_ordinal(string):
    a = []
    aa = []
    num_index = []
    for i in string:
        aa.append(i)
        if i.isdigit():
            a.append(i)
    for i in aa:
        if i.isdigit():
            num_index.append(aa.index(i))
    if a:
        c = ''.join([str(elem) for elem in a])
        cc = int(c)
        b = ordinal(cc)
        beg_ind = num_index[0]
        end_ind = num_index[-1]
        del aa[beg_ind:end_ind+1]
        aa.insert(beg_ind, b)
        bb = ''.join(aa)
        return bb
    else:
        return string

In [11]:
insert_ordinal("57 AV/7 ST")

'577th AV/7 ST'

In [12]:
insert_ordinal("St. George")

'St. George'

In [13]:
mta_stations["STATION_CLEAN"] = mta_stations["STATION_CLEAN"].apply(insert_ordinal)

In [14]:
mta_stations

,STATION,STATION_CLEAN
0,59 ST,59th St
1,5 AV/59 ST,559th St
2,57 ST-7 AV,577th St-7 Av
3,49 ST,49th St
4,TIMES SQ-42 ST,Times Sq-42nd St
...,...,...
374,EASTCHSTER/DYRE,Eastchster-Dyre
375,ST. GEORGE,St. George
376,TOMPKINSVILLE,Tompkinsville
377,RIT-MANHATTAN,Rit-Manhattan


In [15]:
fuzmerge = fuzzy_merge(mta_stations,subway, "STATION_CLEAN", "NAME", threshold=80, limit=1)

In [16]:
fuzmerge.sample(50)

,STATION,STATION_CLEAN,matches
201,PARSONS BLVD,Parsons Blvd,Parsons Blvd
0,59 ST,59th St,59th St
148,ROCKAWAY AV,Rockaway Av,Rockaway Ave
84,SUTTER AV,Sutter Av,Sutter Ave
133,42 ST-PORT AUTH,42nd St-Port Auth,Canal St
80,HALSEY ST,Halsey St,Halsey St
228,NEPTUNE AV,Neptune Av,Neptune Ave
212,47-50 STS ROCK,4750th Sts Rock,50th St
298,SIMPSON ST,Simpson St,Simpson St
127,CATHEDRAL PKWY,Cathedral Pkwy,Cathedral Pkwy (110th St)


In [17]:
fuzmerge = fuzmerge.sort_values("matches", ascending=False)
fuzmerge

,STATION,STATION_CLEAN,matches
326,ZEREGA AV,Zerega Av,Zerega Ave
217,YORK ST,York St,York St
138,WORLD TRADE CTR,World Trade Ctr,World Trade Center
344,61 ST WOODSIDE,61st St Woodside,Woodside - 61st St
292,WOODLAWN,Woodlawn,Woodlawn
...,...,...,...
248,9TH STREET,9thTh Street,
245,PAVONIA/NEWPORT,Pavonia-Newport,
244,EXCHANGE PLACE,Exchange Place,
243,GROVE STREET,Grove Street,


In [21]:
fuzmerge = df.zipcode.astype(str)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 379 entries, 326 to 240
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   STATION        379 non-null    object
 1   STATION_CLEAN  379 non-null    object
 2   matches        379 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [ ]:
fuzmerge.matches.nunique()

In [ ]:
fuzmerge.count()

In [ ]:
len(fuzmerge) - fuzmerge.matches.count()

In [ ]:
fuzmerge.to_pickle("station_key")